In [1]:
from vislearnlabpy.embeddings.generate_embeddings import EmbeddingGenerator
from vislearnlabpy.embeddings.embedding_store import EmbeddingStore
from vislearnlabpy.embeddings.utils import display_search_results, zscore_embeddings, filter_embeddings 
import numpy as np

/home/tsepuri/miniconda3/envs/vislearnlabpy/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


## Loading CLIP embeddings
First let's load our CLIP embedding generator. You can modify the device you're using here. By default this uses cpu or cuda:0 if available.

In [2]:
clip_generator = EmbeddingGenerator(model_type="clip", device=None)

Example 1: First, a simple example with just the embeddings from a directory of images saved within a csv.

In [3]:
clip_generator.generate_image_embeddings(input_dir="input")

Calculating clip embeddings: 100%|██████████| 4/4 [00:00<00:00, 11.53it/s]
Calculating text embeddings: 0it [00:00, ?it/s]


You should be able to see the output embeddings in the output subfolder! You'll also notice that our batch size is currently set to 1. By default, this is saved as a csv with embeddings listed. However, you can specify an output folder, the batch size etc. and save as a docarray or as numpy arrays. Update the values below to reflect in the rest of the file (after example 2). You can leave these as they are too for default behavior.

In [5]:
# Update these here to reflect in the rest of the file. You can leave these as they are for default behavior
OUTPUT_DIRECTORY = None
INPUT_CSV = None # for example: /ccn2/dataset/babyview/outputs_20250312/yoloe_cdi_10k_cropped_by_class_filtered-by-size-0.05/filtered_dataset.csv
INPUT_DIRECTORY = None
OVERWRITE = True
BATCH_SIZE = 1000 # how many images are being processed in one go
SAVE_EVERY_BATCH = False
OUTPUT_TYPE = "doc" # options are "doc" for docarray, "npy" for numpys, "csv" for numbers in a csv

Example 2: Using a CSV file with image paths. To make full use of the pipeline, we recommend saving the embeddings using docarray. Both examples are shown below.

In [5]:
EmbeddingGenerator(output_type="npy").generate_image_embeddings(input_csv="input/inputs.csv", overwrite=OVERWRITE, batch_size=BATCH_SIZE, save_every_batch=SAVE_EVERY_BATCH)

Calculating text embeddings: 100%|██████████| 2/2 [00:00<00:00, 47.32it/s]


In [6]:
EmbeddingGenerator(output_type="doc").generate_image_embeddings(input_csv="input/inputs.csv", overwrite=OVERWRITE, batch_size=BATCH_SIZE, save_every_batch=SAVE_EVERY_BATCH)

Calculating text embeddings: 100%|██████████| 2/2 [00:00<00:00, 55.31it/s]


You should see both the npy csv and the docs file also in the output folder!

You can also convert a npy csv into an embedding store, which we'll be using to search through and filter through embeddings.

In [3]:
embedding_store = EmbeddingStore.from_csv("output/image_embeddings/clip_image_embeddings_npy.csv")
embedding_store.EmbeddingList[0]

100%|██████████| 2/2 [00:00<00:00, 3142.98it/s]


📄 CLIPImageEmbedding : 36b1c5f ...
╭─────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────╮
│ Attribute               │ Value                                                                                 │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ embedding: ImageNdArray │ ImageNdArray of shape (512,), dtype: float16                                          │
│ url: ImageUrl           │ input/acorns/acorn.png                                                                │
│ text: str               │ acorn                                                                                 │
╰─────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────╯

All you now! The rest of the examples below will reflect the store created below.

In [6]:
if INPUT_DIRECTORY is None and INPUT_CSV is None:
    INPUT_CSV = "input/inputs.csv"

In [7]:
EmbeddingGenerator(output_type="doc").generate_image_embeddings(input_csv=INPUT_CSV, batch_size=BATCH_SIZE,
                                                                output_path=OUTPUT_DIRECTORY, input_dir=INPUT_DIRECTORY,
                                                                save_every_batch=SAVE_EVERY_BATCH, overwrite=OVERWRITE)

Calculating text embeddings: 100%|██████████| 2/2 [00:00<00:00, 49.40it/s]


Creating a store from the created embedding docarray (you might have to change the path here depending on your parameters)

In [8]:
embedding_store = EmbeddingStore.from_doc("output/image_embeddings/clip_image_embeddings_doc.docs")

/home/tsepuri/miniconda3/envs/vislearnlabpy/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

## Searching through embeddings

In [10]:
docs, scores = embedding_store.search_store(text_query="spoon", limit=10, categories=["spoon"])
display_search_results(docs, scores)

Error loading None: 'NoneType' object has no attribute 'read'


## Normalize embeddings using z-score

In [ ]:
embedding_store.EmbeddingList.normed_embedding = zscore_embeddings(np.stack(embedding_store.EmbeddingList.embedding))

## Filtering images based on alignment with text.

In [ ]:
filtered_docs = filter_embeddings(embedding_store, alignment_val=0.26)
filtered_store = EmbeddingStore(EmbeddingList=filtered_docs)
print(f"Original file size: {len(embedding_store.EmbeddingList)}")
print(f"New file size: {len(filtered_store.EmbeddingList)}")